In [1]:
from randomgen import RandomNetworkGenerator
import json
import requests
import time

In [2]:
schema = json.load(open("../../../metadata/relations.json", "r"))
generator = RandomNetworkGenerator(schema)

custom_counts = {
    "BusinessUnit": 1,
    "ProductFamily": 2,
    "ProductOffering": 3,
    "Facility": 4,
    "Parts": 5,
    "Warehouse": 6,
    "Supplier": 7,
}

In [3]:
# API_URL = "https://bug-hardy-obviously.ngrok-free.app"
API_URL = "http://localhost:8000"

version = "test17"

In [ ]:
create_ops, graph = generator.create_network(custom_counts)

print(f"Generated {len(create_ops)} operations")

from networkx.readwrite import json_graph
print(json_graph.node_link_data(graph[0], edges="links"))

direct_create_payload = {
    "version": version,
    "action": "direct_create",
    "type": "schema",
    "timestamp": create_ops[0]["timestamp"],
    "payload": json_graph.node_link_data(graph[0], edges="links")
}

requests.post(f"{API_URL}/schema/live/update", json=direct_create_payload)


In [5]:
# bulk_create_payload = {
#     "version": version,
#     "action": "bulk_create",
#     "type": "schema",
#     "timestamp": create_ops[0]["timestamp"],
#     "payload": []
# }

# for op in create_ops:
#     bulk_create_payload["payload"].append(op["payload"])

# print(bulk_create_payload)

# requests.post(f"{API_URL}/schema/live/update", json=bulk_create_payload)

In [ ]:
update_cycles = 5
for _ in range(update_cycles):
    update_ops = generator.generate_updates(
        node_updates=10,
        edge_updates=10
    )
    print(f"Generated {len(update_ops)} update operations")

    bulk_update_payload = {
        "version": version,
        "action": "bulk_update",
        "type": "schema",
        "timestamp": update_ops[0]["timestamp"],
        "payload": []
    }

    for op in update_ops:
        bulk_update_payload["payload"].append(op["payload"])

    print(bulk_update_payload)

    requests.post(f"{API_URL}/schema/live/update", json=bulk_update_payload)
    time.sleep(1)

In [7]:
delete_cycles = 0
for _ in range(delete_cycles):
    delete_ops = generator.generate_deletions(
        node_deletions=5,
        edge_deletions=5
    )
    print(f"Generated {len(delete_ops)} delete operations")

    updated_delete_ops = []

    for op in delete_ops:
        op["version"] = version
        updated_delete_ops.append(op)

    requests.post(f"{API_URL}/schema/live/update/bulk", json=updated_delete_ops)

In [3]:
schema = json.load(open("../../../metadata/relations.json", "r"))
generator = RandomNetworkGenerator(schema)

custom_counts = {
    "BusinessUnit": 1,
    "ProductFamily": 2,
    "ProductOffering": 3,
    "Facility": 4,
    "Parts": 5,
    "Warehouse": 6,
    "Supplier": 7,
}

# API_URL = "https://bug-hardy-obviously.ngrok-free.app"
API_URL = "http://localhost:8000"

version = "test181"

create_ops, graph = generator.create_network(custom_counts)

print(f"Generated {len(create_ops)} operations")

from networkx.readwrite import json_graph
print(json_graph.node_link_data(graph[0], edges="links"))

import os
import sys
sys.path.append("../../../")
from workers import direct_create

direct_create(payload=json_graph.node_link_data(graph[0], edges="links"), timestamp = create_ops[0]["timestamp"], version = version)
print("Direct create complete")

update_cycles = 5
for _ in range(update_cycles):
    update_ops = generator.generate_updates(
        node_updates=10,
        edge_updates=10
    )
    print(f"Generated {len(update_ops)} update operations")

    bulk_update_payload = {
        "version": version,
        "action": "bulk_update",
        "type": "schema",
        "timestamp": update_ops[0]["timestamp"],
        "payload": []
    }

    for op in update_ops:
        bulk_update_payload["payload"].append(op["payload"])

    print(bulk_update_payload)

    requests.post(f"{API_URL}/schema/live/update", json=bulk_update_payload)
    time.sleep(1)

Generated 682 operations
{'directed': True, 'multigraph': False, 'graph': {}, 'nodes': [{'id': '1', 'name': 'Name_8757', 'description': 'Description_3533', 'type': 'Type C', 'cost': 8388.05, 'importance': 1, 'expected_life': 1415, 'units_in_chain': 15, 'expiry': 257, 'node_type': 'Parts', 'usage': 'core'}, {'id': '2', 'name': 'Name_9612', 'description': 'Description_1821', 'type': 'Type C', 'cost': 6878.65, 'importance': 3, 'expected_life': 2911, 'units_in_chain': 83, 'expiry': 75, 'node_type': 'Parts', 'usage': 'core'}, {'id': '3', 'name': 'Name_4565', 'description': 'Description_1892', 'type': 'Type B', 'cost': 2912.05, 'importance': 4, 'expected_life': 3178, 'units_in_chain': 45, 'expiry': 229, 'node_type': 'Parts', 'usage': 'core'}, {'id': '4', 'name': 'Name_9202', 'description': 'Description_3450', 'type': 'Type C', 'cost': 7935.37, 'importance': 1, 'expected_life': 2608, 'units_in_chain': 23, 'expiry': 286, 'node_type': 'Parts', 'usage': 'core'}, {'id': '5', 'name': 'Name_8857', 